### Stage three: Big Data Project

Team Members: <br>
Sumedh Pb - 01FB16ECS403 <br>
Tanmaya Udupa - 01FB16ECS416<br>
Siddhart Itagi - 01FB16ECS382<br>
Sumanth V Rao - 01FB16ECS402 <br>

In [67]:
from pyspark.sql.functions import *
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time
from pyspark import SparkConf,SparkContext
from pyspark.mllib.regression import LabeledPoint
from numpy import array
import math
import random

In [2]:
df = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/over_2016_17.csv")
cluster_batsman = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/cluster_batsman.csv")
cluster_bowler = spark.read.options(header="true",
                               inferSchema="true",
                               nullValue="-",
                               mode="failfast").csv("file:///home/sumanthvrao/bigdata/cluster_bowler.csv")

In [3]:
x = cluster_batsman.groupBy('prediction').mean().cache()
x =  x.withColumnRenamed("avg(Innings_batted)", "Innings_batted").withColumnRenamed("avg(Runs)", "Runs").withColumnRenamed("avg(Batting_strike_rate)", "Batting_strike_rate")
x=x.select("prediction","Innings_batted","Runs", "Batting_strike_rate")
x.show()

+----------+------------------+------------------+-------------------+
|prediction|    Innings_batted|              Runs|Batting_strike_rate|
+----------+------------------+------------------+-------------------+
|         1|114.42857142857143|3347.5714285714284| 136.80142756870814|
|         6| 76.72727272727273|            1953.0| 126.44182031804866|
|         3|              35.0| 605.4545454545455| 126.99136387218128|
|         5|              44.0| 918.0555555555555| 129.49055523342557|
|         9|              24.0|320.16279069767444|  121.5332558210506|
|         4|             100.0| 2511.181818181818| 129.68545393510297|
|         8| 5.423312883435583| 31.33128834355828| 105.43000009454832|
|         7| 11.85135135135135|139.43243243243242|  120.4589192158467|
|        10|137.33333333333334|4027.3333333333335| 133.57666524251303|
|         2|           65.6875|           1330.25|  123.2662501335144|
|         0|1.9895833333333333|3.2395833333333335| 22.887812554836273|
+-----

In [4]:
y = cluster_bowler.groupby("prediction").mean().cache()
y = y.withColumnRenamed("avg(Innings_bowled)", "Innings_bowled").withColumnRenamed("avg(Wickets)", "Wickets").withColumnRenamed("avg(Average)", "Average").withColumnRenamed("avg(Economy)", "Economy")
y = y.select("prediction","Innings_bowled","Wickets", "Average","Economy")
y.show()

+----------+--------------------+-------------------+-------------------+-------------------+
|prediction|      Innings_bowled|            Wickets|            Average|            Economy|
+----------+--------------------+-------------------+-------------------+-------------------+
|         1|0.018867924528301886|                0.0|                0.0|0.05660377358490566|
|         6|   5.384615384615385| 4.3076923076923075|  29.31846178494967| 7.8884615164536696|
|         3|                 5.2|                1.6|               85.8|  9.347999954223633|
|         5|   5.083333333333333|  6.666666666666667|  17.50833336512248|   7.37416672706604|
|         9|  12.333333333333334|  8.416666666666666|   37.4858333269755| 7.9700000286102295|
|         4|  2.2857142857142856|0.09523809523809523|0.19047619047619047|  9.701904751005626|
|         8|   13.31578947368421| 15.105263157894736| 25.013157593576533|  7.942631545819734|
|         7|   5.333333333333333| 2.8333333333333335|  53.15

In [5]:
df.show()

+----+----+------------+------------+-----------+----+-------+
|over|date|     batsman| non_striker|     bowler|runs|wickets|
+----+----+------------+------------+-----------+----+-------+
|   0|2017|    S Dhawan|   DA Warner|   TS Mills|   4|      0|
|   1|2017|MC Henriques|    S Dhawan|A Choudhary|  15|      1|
|   2|2017|MC Henriques|    S Dhawan|   TS Mills|   6|      0|
|   3|2017|    S Dhawan|MC Henriques|  YS Chahal|   4|      0|
|   4|2017|MC Henriques|    S Dhawan|  S Aravind|   9|      0|
|   5|2017|    S Dhawan|MC Henriques|  SR Watson|  17|      0|
|   6|2017|    S Dhawan|MC Henriques|  YS Chahal|   5|      0|
|   7|2017|    S Dhawan|MC Henriques|    TM Head|  11|      0|
|   8|2017|MC Henriques|    S Dhawan|  YS Chahal|   9|      0|
|   9|2017|    S Dhawan|MC Henriques|A Choudhary|   4|      0|
|  10|2017|Yuvraj Singh|MC Henriques|  STR Binny|  10|      1|
|  11|2017|Yuvraj Singh|MC Henriques|  S Aravind|   8|      0|
|  12|2017|Yuvraj Singh|MC Henriques|A Choudhary|  17| 

In [6]:
L = df.rdd.collect()

newL=[]
for row in L:
    
    batsman = cluster_batsman.where(cluster_batsman.Player==row["batsman"])
    #batsman.show()
    try:
        bats_c_no = batsman.select("prediction").collect()[0][0]
    except:
        bats_c_no = -1
    
    nbatsman = cluster_batsman.where(cluster_batsman.Player==row["non_striker"])
    #nbatsman.show()
    try:
        nbats_c_no = nbatsman.select("prediction").collect()[0][0]
    except:
        nbats_c_no = -1
    
    bowler = cluster_bowler.where(cluster_bowler.Player==row["bowler"])
    #bowler.show()
    try:
        bowl_c_no = bowler.select("prediction").collect()[0][0]
    except:
       bowl_c_no = -1
    
    temp=[0 for x in range(7)]
    temp[0]=row[0]
    temp[1]=row[1]
    temp[2]=bats_c_no
    temp[3]=nbats_c_no
    temp[4]=bowl_c_no
    temp[5]=row[5]
    temp[6]=row[6]
    
    newL.append(temp)
print(newL)

[[0, 2017, 1, 1, -1, 4, 0], [1, 2017, 3, 1, -1, 15, 1], [2, 2017, 3, 1, -1, 6, 0], [3, 2017, 1, 3, 8, 4, 0], [4, 2017, 3, 1, 5, 9, 0], [5, 2017, 1, 3, 8, 17, 0], [6, 2017, 1, 3, 8, 5, 0], [7, 2017, 1, 3, 1, 11, 0], [8, 2017, 3, 1, 8, 9, 0], [9, 2017, 1, 3, -1, 4, 0], [10, 2017, 4, 3, 2, 10, 1], [11, 2017, 4, 3, 5, 8, 0], [12, 2017, 4, 3, -1, 17, 0], [13, 2017, 3, 4, 8, 8, 0], [14, 2017, 3, 4, 5, 19, 0], [15, 2017, 9, 4, 8, 4, 1], [16, 2017, 9, 4, -1, 7, 0], [17, 2017, 9, 4, -1, 15, 0], [18, 2017, 9, 8, -1, 12, 1], [19, 2017, 8, 9, 8, 16, 0], [0, 2017, 1, 5, 5, 1, 0], [0, 2017, 5, 1, 5, 10, 0], [1, 2017, 5, 1, 8, 1, 0], [2, 2017, 1, 5, 5, 16, 0], [3, 2017, 5, 1, 8, 14, 0], [4, 2017, 1, 5, 5, 4, 0], [5, 2017, 8, 1, -1, 6, 1], [6, 2017, 3, 8, 6, 7, 1], [7, 2017, 3, 8, -1, 13, 0], [8, 2017, 3, 8, 5, 10, 0], [9, 2017, 3, 8, 9, 12, 0], [10, 2017, 3, 8, 5, 12, 0], [11, 2017, 8, 4, 9, 7, 1], [12, 2017, 4, 7, -1, 8, 1], [13, 2017, 3, 4, 0, 4, 1], [14, 2017, 4, 3, -1, 9, 0], [15, 2017, 4, 3, 5, 

In [7]:
trainData = spark.createDataFrame(newL).withColumnRenamed("_1","overs").withColumnRenamed("_2","year").withColumnRenamed("_3","striker").withColumnRenamed("_4","non_striker").withColumnRenamed("_5","bowler").withColumnRenamed("_6","runs").withColumnRenamed("_7","wickets")

In [8]:
trainData.show()

+-----+----+-------+-----------+------+----+-------+
|overs|year|striker|non_striker|bowler|runs|wickets|
+-----+----+-------+-----------+------+----+-------+
|    0|2017|      1|          1|    -1|   4|      0|
|    1|2017|      3|          1|    -1|  15|      1|
|    2|2017|      3|          1|    -1|   6|      0|
|    3|2017|      1|          3|     8|   4|      0|
|    4|2017|      3|          1|     5|   9|      0|
|    5|2017|      1|          3|     8|  17|      0|
|    6|2017|      1|          3|     8|   5|      0|
|    7|2017|      1|          3|     1|  11|      0|
|    8|2017|      3|          1|     8|   9|      0|
|    9|2017|      1|          3|    -1|   4|      0|
|   10|2017|      4|          3|     2|  10|      1|
|   11|2017|      4|          3|     5|   8|      0|
|   12|2017|      4|          3|    -1|  17|      0|
|   13|2017|      3|          4|     8|   8|      0|
|   14|2017|      3|          4|     5|  19|      0|
|   15|2017|      9|          4|     8|   4|  

In [9]:
new_trainData = trainData.join(x.withColumnRenamed("prediction", "striker").withColumnRenamed("runs", "notimp"),["striker"],"inner").withColumnRenamed("Innings_batted","striker_Innings_batted").withColumnRenamed("Batting_strike_rate","striker_batting_striker_rate").withColumnRenamed("runs","prediction_runs")

In [10]:
new2_trainData= new_trainData.join(x.withColumnRenamed("prediction", "non_striker"),["non_striker"],"inner").withColumnRenamed("Innings_batted","non_striker_Innings_batted").withColumnRenamed("Batting_strike_rate","non_striker_batting_striker_rate")

In [11]:
new3_trainData = new2_trainData.join(y.withColumnRenamed("prediction", "bowler").withColumnRenamed("wickets","overall_wickets"),["bowler"],"inner").withColumnRenamed("wickets","prediction_wickets")

In [12]:
runs_trainData = new3_trainData.select("striker","non_striker","bowler","striker_Innings_batted","striker_batting_striker_rate","non_striker_Innings_batted","non_striker_batting_striker_rate","Innings_bowled","overall_wickets","Average","Economy","prediction_runs")

In [13]:
wickets_trainData = new3_trainData.select("striker","non_striker","bowler","striker_Innings_batted","striker_batting_striker_rate","non_striker_Innings_batted","non_striker_batting_striker_rate","Innings_bowled","overall_wickets","Average","Economy","prediction_wickets")

In [14]:
runs_trainData.show()

+-------+-----------+------+----------------------+----------------------------+--------------------------+--------------------------------+--------------------+------------------+------------------+-------------------+---------------+
|striker|non_striker|bowler|striker_Innings_batted|striker_batting_striker_rate|non_striker_Innings_batted|non_striker_batting_striker_rate|      Innings_bowled|   overall_wickets|           Average|            Economy|prediction_runs|
+-------+-----------+------+----------------------+----------------------------+--------------------------+--------------------------------+--------------------+------------------+------------------+-------------------+---------------+
|      1|          3|     8|    114.42857142857143|          136.80142756870814|                      35.0|              126.99136387218128|   13.31578947368421|15.105263157894736|25.013157593576533|  7.942631545819734|              4|
|      3|          1|     5|                  35.0|     

In [15]:
runs_trainData.printSchema()

root
 |-- striker: long (nullable = true)
 |-- non_striker: long (nullable = true)
 |-- bowler: long (nullable = true)
 |-- striker_Innings_batted: double (nullable = true)
 |-- striker_batting_striker_rate: double (nullable = true)
 |-- non_striker_Innings_batted: double (nullable = true)
 |-- non_striker_batting_striker_rate: double (nullable = true)
 |-- Innings_bowled: double (nullable = true)
 |-- overall_wickets: double (nullable = true)
 |-- Average: double (nullable = true)
 |-- Economy: double (nullable = true)
 |-- prediction_runs: long (nullable = true)



In [16]:
wickets_trainData.printSchema()

root
 |-- striker: long (nullable = true)
 |-- non_striker: long (nullable = true)
 |-- bowler: long (nullable = true)
 |-- striker_Innings_batted: double (nullable = true)
 |-- striker_batting_striker_rate: double (nullable = true)
 |-- non_striker_Innings_batted: double (nullable = true)
 |-- non_striker_batting_striker_rate: double (nullable = true)
 |-- Innings_bowled: double (nullable = true)
 |-- overall_wickets: double (nullable = true)
 |-- Average: double (nullable = true)
 |-- Economy: double (nullable = true)
 |-- prediction_wickets: long (nullable = true)



In [17]:
runs_trainData = runs_trainData.collect()
wickets_trainData = wickets_trainData.collect()

In [18]:
new_runs_data=[]
for row in runs_trainData:
    temp=[0 for x in range(12)]
    temp[0]=row[0]
    temp[1]=row[1]
    temp[2]=row[2]
    temp[3]=row[3]
    temp[4]=row[4]
    temp[5]=row[5]
    temp[6]=row[6]
    temp[7]=row[7]
    temp[8]=row[8]
    temp[9]=row[9]
    temp[10]=row[10]
    temp[11]=row[11]
    new_runs_data.append(temp)
    
new_wickets_data=[]
for row in wickets_trainData:
    temp=[0 for x in range(12)]
    temp[0]=row[0]
    temp[1]=row[1]
    temp[2]=row[2]
    temp[3]=row[3]
    temp[4]=row[4]
    temp[5]=row[5]
    temp[6]=row[6]
    temp[7]=row[7]
    temp[8]=row[8]
    temp[9]=row[9]
    temp[10]=row[10]
    temp[11]=row[11]
    new_wickets_data.append(temp)

In [19]:
print(new_runs_data[0])

[1, 3, 8, 114.42857142857143, 136.80142756870814, 35.0, 126.99136387218128, 13.31578947368421, 15.105263157894736, 25.013157593576533, 7.942631545819734, 4]


In [20]:
def createLabeledPoint(il):
    return LabeledPoint(il[11],array([il[0],il[1],il[2],il[3],il[4],il[5],il[6],il[7],il[8],il[9],il[10]]))
    
runs_training = sc.parallelize(new_runs_data)
trainingRuns = runs_training.map(createLabeledPoint) 

In [21]:
wickets_training = sc.parallelize(new_wickets_data)
trainingWickets = wickets_training.map(createLabeledPoint)

In [62]:
run_model=DecisionTree.trainRegressor(trainingRuns,categoricalFeaturesInfo={0:11, 1:11, 2:10},impurity='variance',maxDepth=5,maxBins=32)

In [63]:
wickets_model=DecisionTree.trainClassifier(trainingWickets, numClasses=12, categoricalFeaturesInfo={0:11, 1:11, 2:10},impurity='gini',maxDepth=5,maxBins=32)

In [90]:
print('Learned Decision tree for runs')
print(run_model.toDebugString())

Learned Decision tree for runs
DecisionTreeModel regressor of depth 5 with 63 nodes
  If (feature 1 in {0.0,5.0,1.0,6.0,2.0,7.0,3.0,4.0})
   If (feature 2 in {9.0,8.0,2.0,5.0})
    If (feature 0 in {0.0,4.0,3.0,5.0})
     If (feature 0 in {0.0})
      If (feature 1 in {7.0,2.0,4.0})
       Predict: 3.8
      Else (feature 1 not in {7.0,2.0,4.0})
       Predict: 6.777777777777778
     Else (feature 0 not in {0.0})
      If (feature 1 in {0.0,5.0,1.0})
       Predict: 6.508670520231214
      Else (feature 1 not in {0.0,5.0,1.0})
       Predict: 7.1440677966101696
    Else (feature 0 not in {0.0,4.0,3.0,5.0})
     If (feature 1 in {0.0,1.0,4.0})
      If (feature 0 in {8.0,1.0,9.0})
       Predict: 6.974545454545455
      Else (feature 0 not in {8.0,1.0,9.0})
       Predict: 7.647368421052631
     Else (feature 1 not in {0.0,1.0,4.0})
      If (feature 0 in {10.0,1.0,9.0,2.0,7.0})
       Predict: 7.568014705882353
      Else (feature 0 not in {10.0,1.0,9.0,2.0,7.0})
       Predict: 8.2540

In [64]:
print('Learned Decision tree for Wickets')
print(wickets_model.toDebugString())

Learned Decision tree for Wickets
DecisionTreeModel classifier of depth 5 with 63 nodes
  If (feature 0 in {5.0,10.0,1.0,6.0,4.0})
   If (feature 1 in {1.0,7.0,0.0})
    If (feature 0 in {10.0,1.0,5.0})
     If (feature 2 in {3.0,7.0,5.0})
      If (feature 1 in {0.0,1.0})
       Predict: 0.0
      Else (feature 1 not in {0.0,1.0})
       Predict: 0.0
     Else (feature 2 not in {3.0,7.0,5.0})
      If (feature 2 in {6.0,9.0,8.0,4.0})
       Predict: 0.0
      Else (feature 2 not in {6.0,9.0,8.0,4.0})
       Predict: 0.0
    Else (feature 0 not in {10.0,1.0,5.0})
     If (feature 2 in {0.0,2.0,7.0,3.0,4.0})
      If (feature 2 in {0.0,2.0,3.0,4.0})
       Predict: 0.0
      Else (feature 2 not in {0.0,2.0,3.0,4.0})
       Predict: 0.0
     Else (feature 2 not in {0.0,2.0,7.0,3.0,4.0})
      If (feature 8 <= 4.3076923076923075)
       Predict: 0.0
      Else (feature 8 > 4.3076923076923075)
       Predict: 0.0
   Else (feature 1 not in {1.0,7.0,0.0})
    If (feature 2 in {1.0,9.0,2.0,7.

In [26]:
testCandidates = [array([1.0,1.0,9.0,35.0,100.99136387218128,20.42857142857143,50.80142756870814,5.083333333333333,6.666666666666667,17.50833336512248,7.37416672706604])]
testData = sc.parallelize(testCandidates)

predictions = run_model.predict(testData)
print("Runs predictions: ")

results = predictions.collect()
for result in results:
    print(result)

Runs predictions: 
9.0


# Driver Function

In [87]:
def driver2(batsman_1,bowler_2,batsman_2,bowler_1):

        score_2=0
        wicket_2=0
        print("-------Innings Ended ----------")
        
        over=0
    
        i=0 #i is index of batsman
        j=1 #next_striker
        k=0 #bowler index

        s_batsman = batsman_2[0]
        ns_batsman = batsman_2[1]
        bowler=bowler_1[0]

        while(j<len(batsman_2) and i<len(batsman_2) and k<len(bowler_1)):

            batsman = cluster_batsman.where(cluster_batsman.Player==s_batsman)
            try:
                bats_c_no = batsman.select("prediction").collect()[0][0]
            except:
                bats_c_no = -1

            nbatsman = cluster_batsman.where(cluster_batsman.Player==ns_batsman)
            try:
                nbats_c_no = nbatsman.select("prediction").collect()[0][0]
            except:
                nbats_c_no = -1

            nbowler = cluster_bowler.where(cluster_bowler.Player==bowler)
            try:
               bowl_c_no = nbowler.select("prediction").collect()[0][0]
            except:
               bowl_c_no = -1

            test_array = [bats_c_no,nbats_c_no,bowl_c_no]

            striker = x.where(x.prediction==bats_c_no)

            try:
                striker_data = striker.select("Innings_batted","Batting_strike_rate").collect()
                test_array.append(striker_data[0][0])
                test_array.append(striker_data[0][1])

            except:
                test_array.append(100)
                test_array.append(130)

            non_striker = x.where(x.prediction==nbats_c_no)

            try:
                nonstriker_data = non_striker.select("Innings_batted","Batting_strike_rate").collect()
                test_array.append(nonstriker_data[0][0])
                test_array.append(nonstriker_data[0][1])

            except:
                test_array.append(100)
                test_array.append(130)


            bowler_d= y.where(y.prediction==bowl_c_no)

            try:
                bowler_data = bowler_d.select("Innings_bowled","Wickets","Average","Economy").collect()
                test_array.append(bowler_data[0][0])
                test_array.append(bowler_data[0][1])
                test_array.append(bowler_data[0][2])
                test_array.append(bowler_data[0][3])

            except:
                test_array.append(7)
                test_array.append(10)
                test_array.append(15)
                test_array.append(8)


            testCandidates = [array(test_array)]
            testData = sc.parallelize(testCandidates)

            prediction_runs = run_model.predict(testData)
            prediction_wicket = wickets_model.predict(testData)
                    
            result3 = prediction_runs.collect()
            result4 = prediction_wicket.collect()
            for ele in result3:
                score_2+=math.floor(ele)
            for ele in result4:
                wicket_2+=math.floor(ele)
                if(wicket_2>0):
                    k+=1
                    i+=1
                    if(i>=len(batsman_2) or k>=len(bowler_1)):
                        print(str(score_2)+"/"+str(wicket_2))
                        return 
                    #print(s_batsman,ns_batsman,bowler)
                    s_batsman = batsman_2[i]
                    ns_batsman = batsman_2[j]
                    bowler=bowler_1[k]
                else:
                    k+=1
                    (s_batsman,ns_batsman)=(ns_batsman,s_batsman)
                    if(k>=len(bowler_2)):
                        print(str(score_2)+"/"+str(wicket_2))
                        return 
                    bowler=bowler_1[k]
            print(str(score_2)+"/"+str(wicket_2))
            
            
#-----------------------------------------------------------------------------------
def driver(batsman_1,bowler_2,batsman_2,bowler_1):
    score_1=0
    wicket_1=0
    over=0
    
    i=0 #i is index of batsman
    j=1 #next_striker
    k=0 #bowler index
    
    s_batsman = batsman_1[0]
    ns_batsman = batsman_1[1]
    bowler=bowler_2[0]
        
    while(j<len(batsman_1) and i<len(batsman_1) and k<len(bowler_2)):
        batsman = cluster_batsman.where(cluster_batsman.Player==s_batsman)
        try:
            bats_c_no = batsman.select("prediction").collect()[0][0]
        except:
            bats_c_no = -1
            
        nbatsman = cluster_batsman.where(cluster_batsman.Player==ns_batsman)
        try:
            nbats_c_no = nbatsman.select("prediction").collect()[0][0]
        except:
            nbats_c_no = -1
        
        nbowler = cluster_bowler.where(cluster_bowler.Player==bowler)
        try:
           bowl_c_no = nbowler.select("prediction").collect()[0][0]
        except:
           bowl_c_no = -1
        
        test_array = [bats_c_no,nbats_c_no,bowl_c_no]
        
        striker = x.where(x.prediction==bats_c_no)
        
        try:
            striker_data = striker.select("Innings_batted","Batting_strike_rate").collect()
            test_array.append(striker_data[0][0])
            test_array.append(striker_data[0][1])
        
        except:
            test_array.append(100)
            test_array.append(130)
            
        non_striker = x.where(x.prediction==nbats_c_no)
        
        try:
            nonstriker_data = non_striker.select("Innings_batted","Batting_strike_rate").collect()
            test_array.append(nonstriker_data[0][0])
            test_array.append(nonstriker_data[0][1])
        
        except:
            test_array.append(100)
            test_array.append(130)
            
        
        bowler_d= y.where(y.prediction==bowl_c_no)
        
        try:
            bowler_data = bowler_d.select("Innings_bowled","Wickets","Average","Economy").collect()
            test_array.append(bowler_data[0][0])
            test_array.append(bowler_data[0][1])
            test_array.append(bowler_data[0][2])
            test_array.append(bowler_data[0][3])
        
        except:
            test_array.append(7)
            test_array.append(10)
            test_array.append(15)
            test_array.append(8)
        
        
        testCandidates = [array(test_array)]
        testData = sc.parallelize(testCandidates)
        
        prediction_runs = run_model.predict(testData)
        prediction_wicket = wickets_model.predict(testData)
        
        
                
        result1 = prediction_runs.collect()
        result2 = prediction_wicket.collect()
        for ele in result1:
            score_1+=math.ceil(ele)
        for ele in result2:
            wicket_1+=math.ceil(ele)
            if(wicket_1>0):
                k+=1
                i+=1
                if(i>=len(batsman_1) or (k>=len(bowler_2))):
                    print(str(score_1)+"/"+str(wicket_1))
                    driver2(batsman_1,bowler_2,batsman_2,bowler_1)
                    print("-------Match Ended-------")
                    return
                #print(s_batsman,ns_batsman,bowler)
                s_batsman = batsman_1[i]
                ns_batsman = batsman_1[j]
                bowler=bowler_2[k]
                 
            else:
                k+=1
                #print(s_batsman,ns_batsman,bowler)
                (s_batsman,ns_batsman)=(ns_batsman,s_batsman)
                if(k>=len(bowler_2)):
                    print(str(score_1)+"/"+str(wicket_1))
                    driver2(batsman_1,bowler_2,batsman_2,bowler_1)
                    print("-------Match Ended-------")
                    return
                bowler=bowler_2[k]
        print(str(score_1)+"/"+str(wicket_1))
        


In [33]:
batsman_1=["RG Sharma","Ishan Kishan"]    
bowler_2=["SR Watson"]
batsman_2=[]
bowler_1=[]
driver(batsman_1,bowler_2,batsman_2,bowler_1)

4.0/0.0


### Match number 1  MI 165/4  CSK 169/9 19.5 overs

In [89]:
batsman_1=[
"RG Sharma",
"Ishan Kishan" 
,"SA Yadav"
,"JC Buttler"
,"N Rana"
,"HH Pandya"
,"KH Pandya"
,"KA Pollard"
"MJ McClenaghan"
,"Mustafizur Rahman"
,"JJ Bumrah"]

bowler_2=[
"SR Watson",
"Imran Tahir",
"SR Watson",
"Imran Tahir",
"DJ Bravo" ,
"Harbhajan Singh",
"DL Chahar",
"Harbhajan Singh",
"RA Jadeja",
"DL Chahar",
"DJ Bravo",
"Harbhajan Singh",
"DL Chahar",
"Imran Tahir",
"DJ Bravo",
"Imran Tahir",
"SR Watson" ,
"Harbhajan Singh",
"DJ Bravo",
"SR Watson"
]

batsman_2=[
"SR Watson",
"AT Rayudu",
"SK Raina",
"KM Jadhav",
"MS Dhoni",
"RA Jadeja",
"DJ Bravo",
"DL Chahar",
"Harbhajan Singh",
"Imran Tahir"]

bowler_1=[
"MJ McClenaghan",
"JJ Bumrah",
"MJ McClenaghan",
"JJ Bumrah",
"KH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"KH Pandya",
"Mustafizur Rahman",
"HH Pandya",
"KH Pandya",
"Mustafizur Rahman",  
"JJ Bumrah",
"KH Pandya",
"HH Pandya",
"MJ McClenaghan",
"JJ Bumrah",
"MJ McClenaghan"  
]

driver(batsman_1,bowler_2,batsman_2,bowler_1)

8/0
16/0
24/0
32/0
40/0
48/0
52/0
60/0
68/0
76/0
84/0
92/0
96/0
104/0
112/0
120/0
128/0
136/0
144/0
152/0
-------Innings Ended ----------
7/0
14/0
21/0
28/0
35/0
42/0
49/0
56/0
63/0
70/0
77/0
84/0
91/0
98/0
105/0
112/0
119/0
126/0
133/0
140/0
-------Match Ended-------


### Match Number 2  DD- 166/7 20 overs KXiP - 167/4 18.5 overs

In [66]:
batsman_1=[
"C Munro",
"G Gambhir",
"SS Iyer",
"V Shankar",
"RR Pant",
"R Tewatia",
"CH Morris",
"DT Christian",
"A Mishra",
"TA Boult",
"Mohammed Shami"]


bowler_2=[
"AR Patel"
,"R Ashwin"
,"Sandeep Sharma"
,"MM Sharma"
,"AB Dinda"
,"AR Patel"
,"R Ashwin"
,"Sandeep Sharma"
,"MM Sharma"
,"AB Dinda"
,"AR Patel"
,"R Ashwin"
,"Sandeep Sharma"
,"MM Sharma"
,"AB Dinda"
,"AR Patel"
,"R Ashwin"
,"Sandeep Sharma"
,"MM Sharma"
,"AB Dinda"
  
]


batsman_2=[
"KL Rahul",
"MA Agarwal" 
,"Yuvraj Singh"
,"KK Nair"
,"DA Miller"
,"MP Stoinis"
,"AR Patel"
,"R Ashwin"
,"Sandeep Sharma"
,"MM Sharma"
,"AB Dinda"] 



bowler_1=[
"CH Morris",
"C Munro",
"A Mishra",
"TA Boult",
"Mohammed Shami",
"CH Morris","C Munro",
"A Mishra",
"TA Boult",
"Mohammed Shami",
"CH Morris",
"C Munro",
"A Mishra",
"TA Boult",
"Mohammed Shami",
"CH Morris",
"C Munro",
"A Mishra",
"TA Boult",
"Mohammed Shami"
]
driver(batsman_1,bowler_2,batsman_2,bowler_1)

7/0
17/0
24/0
34/0
41/0
51/0
58/0
68/0
75/0
85/0
92/0
102/0
109/0
119/0
126/0
136/0
143/0
153/0
160/0
170/0
-------Innings Ended ----------
6/0
14/0
20/0
27/0
35/0
42/0
50/0
57/0
65/0
72/0
78/0
86/0
92/0
99/0
107/0
114/0
122/0
129/0
137/0
144/0


In [86]:
1+2

3